# Testing Deployed Toxic Tweets Detection Pipeline


In [64]:
import tensorflow as tf
import pandas as pd
import requests
import base64
import json
import os
from pprint import PrettyPrinter

pp = PrettyPrinter()

## Checking Model Status


In [7]:
pp.pprint(
    requests.get("http://localhost:8080/v1/models/toxic-tweets-detection-model").json()
)

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1714632557'}]}


## Testing from Dataset


### Loading the Dataset


In [10]:
DATASET_PATH = os.path.abspath("data/data.csv")

df = pd.read_csv(DATASET_PATH)
df.head()

,is_toxic,tweet
0,0,Three little birds
1,0,i like this positive #quote! never stress to t...
2,1,"RT @SoulYodeler: Go ahead and mock me, one day..."
3,0,wh is enemy deployed google for illegal manip....
4,1,I'm happy I ain't that bitch I use to be


### Creating Samples to Test


In [249]:
samples = df.sample(10, ignore_index=True)
texts = samples["tweet"]
labels = samples["is_toxic"]

#### Converting Samples into TF Serving Request Format


In [251]:
def base64_examples(text):
    feature_spec = {
        "tweet": tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])
        )
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    return {"examples": {"b64": base64.b64encode(example).decode()}}

In [252]:
instances = [base64_examples(text) for text in texts]
request_data = json.dumps({"signature_name": "serving_default", "instances": instances})

#### Getting Response


In [253]:
response = requests.post(
    "http://localhost:8081/v1/models/toxic-tweets-detection-model:predict",
    data=request_data,
).json()

#### Converting Predictions Response as Label


In [254]:
threshold = 0.5
pred_labels = [
    "TOXIC" if int(prediction[0] >= threshold) else "NON-TOXIC"
    for prediction in response.get("predictions")
]

#### Creating DataFrame to Visualize Predictions


In [255]:
result_df = pd.DataFrame(
    {
        "text": texts,
        "label": ["TOXIC" if label == 1 else "NON-TOXIC" for label in labels],
        "prediction": pred_labels,
    }
)

result_df

,text,label,prediction
0,@user @user it's a very cruel world we live in...,NON-TOXIC,NON-TOXIC
1,@user as a huge @user fan i'm excited for liv...,NON-TOXIC,NON-TOXIC
2,#prayfororlando #orlandoshooting #orlando ...,NON-TOXIC,NON-TOXIC
3,RT @BrandonTweets__: Who else remembers these ...,TOXIC,TOXIC
4,"RT @StevStiffler: Wtf is ""Lolzzz"" ?! Did you l...",TOXIC,TOXIC
5,10 things that made me happy this week 10 thi...,NON-TOXIC,NON-TOXIC
6,Ohh &amp; btw @Leslieeeixta when you get a cal...,TOXIC,TOXIC
7,Nigga you made it RT @HtownsNuNu: Nigga we mad...,TOXIC,TOXIC
8,happy bihday blue card bihday #blue #handma...,NON-TOXIC,NON-TOXIC
9,Easy bitch how ? Lmfaoooooooooo She was my gir...,TOXIC,TOXIC
